#### Example 11: Evalscript

Instead of using Sentinel Hub Configurator we can define our own custom layers inside Python with ``CustomUrlParam.EVALSCRIPT``. All we need is a chunk of code written in Javascript which is not too long to fit into an URL.

Let's implement a simple cloud detection algorithm.

In [ ]:
# by Braaten, Cohen, Yang 2015
my_evalscript = '''
var bRatio = (B01 - 0.175) / (0.39 - 0.175);
var NGDR = (B01 - B02) / (B01 + B02);

function clip(a) {
    return a>0 ? (a<1 ? a : 1) : 0;
}
    
if (bRatio > 1) {
    var v = 0.5*(bRatio - 1);
    return [0.5*clip(B04), 0.5*clip(B03), 0.5*clip(B02) + v];
}
    
if (bRatio > 0 && NGDR > 0) {
    var v = 5 * Math.sqrt(bRatio * NGDR);
    return [0.5 * clip(B04) + v, 0.5 * clip(B03), 0.5 * clip(B02)];
}
    
return [2*B04, 2*B03, 2*B02];
'''

evalscript_wms_request = WmsRequest(layer='TRUE-COLOR-S2-L1C', # Layer parameter can be any existing layer
                                    bbox=betsiboka_bbox, 
                                    time='2017-12-20', 
                                    width=512,
                                    instance_id=INSTANCE_ID,
                                    custom_url_params={CustomUrlParam.EVALSCRIPT: my_evalscript})

evalscript_wms_data = evalscript_wms_request.get_data()
plot_image(evalscript_wms_data[0])

Note: We still had to specify an existing layer from Configurator. That is because each layer is linked with it's data source and we cannot override layer's data source from the code. 

#### Example 12: Evalscript URL

Another option is to simply provide an URL address of an evalscript written in Javascript. For that purpose we have created a [collection of useful custom scripts on Github](https://github.com/sentinel-hub/custom-scripts).

Let's select a script for calculating moisture index and provide its URL as a value of parameter `CustomUrlParam.EVALSCRIPTURL`.

In [ ]:
my_url = 'https://raw.githubusercontent.com/sentinel-hub/custom-scripts/master/sentinel-2/moisture_index/scripts.js'

evalscripturl_wms_request = WmsRequest(layer='TRUE-COLOR-S2-L1C', # Layer parameter can be any existing layer
                                       bbox=betsiboka_bbox, 
                                       time='2017-12-20', 
                                       width=512,
                                       instance_id=INSTANCE_ID,
                                       custom_url_params={CustomUrlParam.EVALSCRIPTURL: my_url})

evalscripturl_wms_data = evalscripturl_wms_request.get_data()
plot_image(evalscripturl_wms_data[0])

### Data Sources

The package supports various data sources. Default data source is Sentinel-2 L1C however currently the following is supported:

In [ ]:
from sentinelhub import DataSource

for source in DataSource.get_available_sources():
    print(source)

In order to obtain data from any of these data sources with `WmsRequest` or `WcsRequest` we have to do the following: 

- Use  a configuration based on **Python scripts template** or create a new layer in [Sentinel Hub Configurator](https://apps.sentinel-hub.com/configurator/) that is defined to use desired satellite data source. Set the `layer` parameter of `WmsRequest` or `WcsRequest` to the name of this newly created layer.
- Set `data_source` parameter of `WmsRequest` or `WcsRequest` to the same data source (using one of the objects from the list above).

#### Example 13: Sentinel-2 L2A

When you have a layer named `TRUE-COLOR-S2-L2A` in your configuration let's try to obtain some level 2A images. Unfortunately L2A images are being processed only for some regions around the globe and Betsiboka Estuary is not one of them.

Instead let's check [Eyjafjallajökull volcano](https://en.wikipedia.org/wiki/Eyjafjallaj%C3%B6kull) on Iceland. This time we will provide coordinates in Popular Web Mercator CRS.

In [ ]:
volcano_bbox = BBox(bbox=[(-2217485.0, 9228907.0), (-2150692.0, 9284045.0)], crs=CRS.POP_WEB)

l2a_request = WmsRequest(data_source=DataSource.SENTINEL2_L2A,
                         layer='TRUE-COLOR-S2-L2A',
                         bbox=volcano_bbox,
                         time='2017-08-30',
                         width=512,
                         instance_id=INSTANCE_ID)
l2a_data = l2a_request.get_data()
plot_image(l2a_data[0])

#### Example 14: DEM

To check the elevation model of chosen area we need a layer `DEM` defined with the following custom script:

```Javascript
return [DEM]
```

Request using Mapzen DEM as a data source does not require a time parameter.

In [ ]:
dem_request = WmsRequest(data_source=DataSource.DEM,
                         layer='DEM',
                         bbox=volcano_bbox,
                         width=512,
                         instance_id=INSTANCE_ID,
                         image_format=MimeType.TIFF_d32f,
                         custom_url_params={CustomUrlParam.SHOWLOGO: False})

dem_image = dem_request.get_data()[0]

plot_image(dem_image, 1 / np.amax(dem_image))

#### Example 15: Landsat 8

To view Landsat 8 L1C image we require a layer `TRUE-COLOR-L8` with predefined true color RGB template.

In [ ]:
l8_request = WmsRequest(data_source=DataSource.LANDSAT8,
                        layer='TRUE-COLOR-L8',
                        bbox=volcano_bbox,
                        time='2017-08-20',
                        width=512,
                        instance_id=INSTANCE_ID)

l8_data = l8_request.get_data()
plot_image(l8_data[-1])

#### Example 15: Sentinel-1

If we would like to avoid clouds using Sentinel-1 radar data seems like a good idea. The package supports obtaining multiple types of Sentinel-1 data. In this example we will use `DataSource.SENTINEL1_IW`. While creating layer in Sentinel Hub Configurator we must be careful to use the same settings as the supported data source: 

In [ ]:
DataSource.SENTINEL1_IW

This tells us we have to set acquisition parameter to `IW`, polarisation to `DV`, resolution to `High` and orbit direction to `Both`. After that let's name the layer `TRUE-COLOR-S1-IW` and use the following custom script

```Javascript
return [VV, 2 * VH, VV / VH / 100.0]
```

In [ ]:
s1_request = WmsRequest(data_source=DataSource.SENTINEL1_IW,
                         layer='TRUE-COLOR-S1-IW',
                         bbox=volcano_bbox,
                         time='2017-10-03',
                         width=512,
                         instance_id=INSTANCE_ID)

s1_data = s1_request.get_data()
plot_image(s1_data[-1])

#### Example 16: Sentinel-1, ascending orbit direction

Sentinel-1 data is acquired when a satellite travels either from approx. north to south (i.e. `DESCENDING` orbit direction) or from approx. south to north (i.e.`ASCENDING` orbit direction). With sentinelhub-py package one can request only the data with `ASCENDING` orbit direction if using a datasource ending with "\_ASC" (or only data with `DESCENDING` orbit direction when using datasource ending with "\_DES").   

E.g., the request below will fetch only the data with `ASCENDING` orbit direction:

In [ ]:
s1_asc_request = WmsRequest(data_source=DataSource.SENTINEL1_IW_ASC,
                         layer='TRUE-COLOR-S1-IW',
                         bbox=volcano_bbox,
                         time=('2017-10-03','2017-10-05'),
                         width=512,
                         instance_id=INSTANCE_ID)

s1_asc_data = s1_asc_request.get_data()
plot_image(s1_asc_data[-1])

#### Example 17: Custom (BYOC) data

It is possible that you "bring your own data" (BYOC) and access it using Sentinel Hub in a similar manner as any other satellite data. 
To be able to access your own data using Sentinel Hub you will need to prepare a few things. Roughly speaking these are (find all details here: https://docs.sentinel-hub.com/api/latest/#/API/byoc):
- Convert your data to Cloud Optimized GeoTiff (COG) format. Store it in AWS S3 bucket and allow SH to access it.
- Create collection in SH, which points to the S3 bucket. Within the collection, create tiles. 
- Create new layer in your configuration, which points to the collection.
- Request the data using SentinelHub.py package

To demonstrate this, we have prepeared a demo S3 bucket, demo collection and demo layer in our configuration. Tu acess this data we need collection id = '31df1de4-8bd4-43e0-8c3f-b04262d111b6' and layer id = 'DEMO_BYOC_LAYER'. Let see how to request the custom data.

In [ ]:
bbox =  [13.82387,45.85221, 13.83313, 45.85901]
byoc_bbox = BBox(bbox=bbox, crs=CRS.WGS84)

collection_id = '<your collection id>'
layer = 'DEMO_BYOC_LAYER'

byoc_request = WmsRequest(data_source=DataSource(collection_id),
                        layer=layer,
                        bbox=byoc_bbox,
                        width=512,
                        instance_id=INSTANCE_ID)

byoc_data = byoc_request.get_data()
plot_image(byoc_data[-1])